# 🤖 S.A.M Blocks AI Training - Google Colab
## Train YOLOv8 Block Detection Model

**Dataset Location:** `/content/drive/MyDrive/block_dataset`  
**Training Time:** 15-30 minutes with GPU  
**Goal:** Train AI to detect and count blocks in images

---

## ⚡ IMPORTANT: Enable GPU First!
1. Click **Runtime** → **Change runtime type**
2. Select **Hardware accelerator: GPU**
3. Click **Save**

Then run all cells in order (Shift + Enter)

## Step 1: Mount Google Drive 📁
Connect to your Google Drive to access the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")
print("📁 Your dataset is at: /content/drive/MyDrive/block_dataset")

## Step 2: Check GPU ⚡
Verify that GPU is enabled (should show NVIDIA Tesla T4 or similar)

In [ ]:
!nvidia-smi

## Step 3: Install YOLOv8 📦
Install Ultralytics library for training (takes ~2 minutes)

In [ ]:
!pip install ultralytics -q

# Verify installation
import ultralytics
print(f"✅ Ultralytics installed: version {ultralytics.__version__}")

## Step 4: Verify Dataset Structure 📊
Check what's inside your block_dataset folder

In [ ]:
import os
from pathlib import Path

# Your dataset location
dataset_path = Path('/content/drive/MyDrive/block_dataset')

print(f"📁 Dataset location: {dataset_path}")
print(f"✅ Exists: {dataset_path.exists()}\n")

if dataset_path.exists():
    print("📂 Contents:")
    for item in sorted(dataset_path.iterdir()):
        if item.is_dir():
            file_count = len(list(item.iterdir()))
            print(f"  📁 {item.name}/ ({file_count} files)")
        else:
            print(f"  📄 {item.name}")
    
    # Check for images and labels
    images_path = dataset_path / 'images'
    labels_path = dataset_path / 'labels'
    
    if images_path.exists():
        images = list(images_path.glob('*.[jJ][pP]*[gG]')) + list(images_path.glob('*.[pP][nN][gG]'))
        print(f"\n📸 Total images: {len(images)}")
    
    if labels_path.exists():
        labels = list(labels_path.glob('*.txt'))
        print(f"🏷️  Total labels: {len(labels)}")
        
        # Find matched pairs
        if images_path.exists():
            image_stems = {img.stem for img in images}
            label_stems = {lbl.stem for lbl in labels}
            matched = image_stems & label_stems
            print(f"✅ Matched (labeled images): {len(matched)}")
            print(f"⚠️  Unlabeled images: {len(image_stems - label_stems)}")
else:
    print("❌ Dataset not found! Check the path.")

## Step 5: Copy Dataset to Colab Storage 🚀
Copy from Google Drive to Colab's local storage for faster training

In [ ]:
import shutil

# Copy entire dataset to Colab's fast local storage
source = Path('/content/drive/MyDrive/block_dataset')
destination = Path('/content/block_dataset')

if destination.exists():
    shutil.rmtree(destination)

print("📦 Copying dataset to Colab storage...")
shutil.copytree(source, destination)
print(f"✅ Copied to {destination}")

# Verify
images_count = len(list((destination / 'images').glob('*')))
labels_count = len(list((destination / 'labels').glob('*.txt')))
print(f"📸 Images: {images_count}")
print(f"🏷️  Labels: {labels_count}")

## Step 6: Split Train/Validation Data 📊
Create 80% train / 20% validation split

In [ ]:
import random

dataset_path = Path('/content/block_dataset')
images_path = dataset_path / 'images'
labels_path = dataset_path / 'labels'

# Create train/val directories
train_images = dataset_path / 'train' / 'images'
train_labels = dataset_path / 'train' / 'labels'
val_images = dataset_path / 'val' / 'images'
val_labels = dataset_path / 'val' / 'labels'

for p in [train_images, train_labels, val_images, val_labels]:
    p.mkdir(parents=True, exist_ok=True)

# Get all labeled images (images with corresponding .txt labels)
image_files = list(images_path.glob('*.[jJ][pP]*[gG]')) + list(images_path.glob('*.[pP][nN][gG]'))
labeled_images = []

for img in image_files:
    label_file = labels_path / f"{img.stem}.txt"
    if label_file.exists():
        labeled_images.append(img)

print(f"📊 Found {len(labeled_images)} labeled images")

# Shuffle and split (80% train, 20% validation)
random.seed(42)  # For reproducibility
random.shuffle(labeled_images)

split_idx = int(len(labeled_images) * 0.8)
train_imgs = labeled_images[:split_idx]
val_imgs = labeled_images[split_idx:]

print(f"📈 Train: {len(train_imgs)} images")
print(f"📉 Validation: {len(val_imgs)} images")

# Copy files to train/val directories
for img in train_imgs:
    shutil.copy(img, train_images / img.name)
    label = labels_path / f"{img.stem}.txt"
    shutil.copy(label, train_labels / f"{img.stem}.txt")

for img in val_imgs:
    shutil.copy(img, val_images / img.name)
    label = labels_path / f"{img.stem}.txt"
    shutil.copy(label, val_labels / f"{img.stem}.txt")

print("✅ Dataset split complete!")

## Step 7: Create data.yaml Configuration 📝
YOLO needs this config file to find training data

In [ ]:
# Create data.yaml for YOLO training
data_yaml_content = f"""# S.A.M Blocks Dataset Configuration
path: {dataset_path}
train: train/images
val: val/images

names:
  0: block

nc: 1
"""

data_yaml_path = dataset_path / 'data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"✅ Created {data_yaml_path}\n")
print("Contents:")
print(data_yaml_content)

## Step 8: START TRAINING! 🚀🔥
Train YOLOv8 model (15-30 minutes with GPU)

**Watch the progress below:**
- Epochs: 50 training cycles
- Lower loss values = better learning
- Training saves checkpoints every 10 epochs

In [ ]:
from ultralytics import YOLO
import os

# Create directory to save results to Google Drive
save_dir = '/content/drive/MyDrive/sam_blocks_training_results'
os.makedirs(save_dir, exist_ok=True)

# Load pretrained YOLOv8 Nano segmentation model
print("📦 Loading YOLOv8 Nano model...")
model = YOLO('yolov8n-seg.pt')

print("\n🚀 Starting training...")
print("=" * 60)

# Train the model
results = model.train(
    data=str(dataset_path / 'data.yaml'),
    epochs=50,              # Number of training cycles
    imgsz=640,              # Image size (640x640)
    batch=8,                # Batch size (reduce to 4 if out-of-memory)
    patience=10,            # Stop early if no improvement for 10 epochs
    save=True,              # Save checkpoints
    project=save_dir,       # Save to Google Drive
    name='sam_blocks_v1',   # Experiment name
    exist_ok=True,          # Overwrite if exists
    pretrained=True,        # Use pretrained weights
    optimizer='Adam',       # Adam optimizer
    verbose=True,           # Show detailed progress
    seed=42,                # Random seed
    deterministic=True,     # Reproducible training
    single_cls=True,        # Single class (blocks only)
    plots=True,             # Generate training plots
    save_period=10,         # Save checkpoint every 10 epochs
)

print("=" * 60)
print("\n🎉 TRAINING COMPLETE!")
print(f"📁 Results saved to: {save_dir}/sam_blocks_v1")
print(f"🏆 Best model: {save_dir}/sam_blocks_v1/weights/best.pt")

## Step 9: View Training Results 📊
Look at training curves and example detections

In [ ]:
from IPython.display import Image, display
import os

results_dir = f"{save_dir}/sam_blocks_v1"

print("📊 TRAINING RESULTS\n")
print("=" * 60)

# 1. Confusion Matrix
conf_matrix = f"{results_dir}/confusion_matrix.png"
if os.path.exists(conf_matrix):
    print("\n1️⃣ Confusion Matrix:")
    display(Image(filename=conf_matrix, width=600))

# 2. Training Curves (loss, metrics over epochs)
results_img = f"{results_dir}/results.png"
if os.path.exists(results_img):
    print("\n2️⃣ Training Curves (Loss & Metrics):")
    display(Image(filename=results_img, width=900))

# 3. Validation Predictions
val_batch0 = f"{results_dir}/val_batch0_pred.jpg"
if os.path.exists(val_batch0):
    print("\n3️⃣ Example Detections on Validation Images:")
    display(Image(filename=val_batch0, width=900))

# 4. F1 Curve
f1_curve = f"{results_dir}/F1_curve.png"
if os.path.exists(f1_curve):
    print("\n4️⃣ F1 Score Curve:")
    display(Image(filename=f1_curve, width=600))

print("\n" + "=" * 60)

## Step 10: Calculate Model Metrics 📈
See how accurate your model is

In [ ]:
# Load the best trained model
best_model_path = f"{save_dir}/sam_blocks_v1/weights/best.pt"
trained_model = YOLO(best_model_path)

# Run validation to get metrics
print("📊 Calculating metrics on validation set...\n")
metrics = trained_model.val()

print("=" * 60)
print("🎯 MODEL PERFORMANCE METRICS")
print("=" * 60)

# Box detection metrics
print(f"\n📦 Bounding Box Detection:")
print(f"  mAP50 (IoU=0.50):     {metrics.box.map50:.3f}")
print(f"  mAP50-95:             {metrics.box.map:.3f}")
print(f"  Precision:            {metrics.box.mp:.3f}")
print(f"  Recall:               {metrics.box.mr:.3f}")

# Segmentation metrics (if available)
if hasattr(metrics, 'seg'):
    print(f"\n🎭 Segmentation Masks:")
    print(f"  mAP50 (IoU=0.50):     {metrics.seg.map50:.3f}")
    print(f"  mAP50-95:             {metrics.seg.map:.3f}")
    print(f"  Precision:            {metrics.seg.mp:.3f}")
    print(f"  Recall:               {metrics.seg.mr:.3f}")

print("\n" + "=" * 60)
print("\n💡 What these mean:")
print("  • mAP50 > 0.5  = Decent model")
print("  • mAP50 > 0.7  = Good model")
print("  • mAP50 > 0.9  = Excellent model")
print("\n  • Precision = % of detections that are actually blocks")
print("  • Recall    = % of actual blocks that were detected")
print("=" * 60)

## Step 11: Test on Sample Image 🖼️
See your model detect blocks in real-time!

In [ ]:
import matplotlib.pyplot as plt

# Get a validation image
test_image = list(val_images.glob('*'))[0]
print(f"🖼️  Testing on: {test_image.name}\n")

# Run inference
results = trained_model(test_image)

# Display result with detections drawn
result = results[0]
result_image = result.plot()  # Draw boxes and masks

plt.figure(figsize=(14, 10))
plt.imshow(result_image)
plt.axis('off')
plt.title(f"Detection Result: {test_image.name}", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print detection details
print("=" * 60)
print(f"✅ Detected {len(result.boxes)} block(s)")
print("=" * 60)

for i, box in enumerate(result.boxes):
    conf = box.conf[0].item()
    cls = int(box.cls[0].item())
    print(f"  Block {i+1}: {conf:.1%} confidence")

print("=" * 60)

## Step 12: Save Final Model 💾
Copy the trained model to an easy-to-find location in Google Drive

In [ ]:
import shutil

# Copy best model to Drive root for easy download
final_model_src = f"{save_dir}/sam_blocks_v1/weights/best.pt"
final_model_dest = '/content/drive/MyDrive/block_detector_TRAINED.pt'

shutil.copy(final_model_src, final_model_dest)

# Get file size
file_size_mb = os.path.getsize(final_model_dest) / (1024 * 1024)

print("=" * 60)
print("🎉 MODEL SAVED SUCCESSFULLY!")
print("=" * 60)
print(f"\n📦 Model location: block_detector_TRAINED.pt")
print(f"📊 File size: {file_size_mb:.1f} MB")
print(f"\n📥 To download:")
print("  1. Go to https://drive.google.com/")
print("  2. Find 'block_detector_TRAINED.pt' in My Drive")
print("  3. Right-click → Download")
print("  4. Save to your computer")
print("\n📂 Full results folder: sam_blocks_training_results/sam_blocks_v1/")
print("=" * 60)

print("\n✅ ALL DONE! Your AI model is ready to use!")

---

## 🎓 What You Just Did

Congratulations! You just:
- ✅ Trained a deep learning model from scratch
- ✅ Used transfer learning (started with pretrained YOLOv8)
- ✅ Split data scientifically (80/20 train/val)
- ✅ Evaluated model performance with metrics
- ✅ Created a production-ready AI model

## 📊 Understanding Your Results

**If mAP50 is:**
- **0.3 - 0.5**: Basic detection, needs more training data
- **0.5 - 0.7**: Good! Model works well for most cases
- **0.7 - 0.9**: Excellent! Ready for production
- **0.9+**: Outstanding! Professional-grade accuracy

## 🚀 Next Steps

1. **Download the model** (`block_detector_TRAINED.pt`) from Google Drive
2. **Add to your project:**
   ```bash
   # On your computer:
   Move-Item "C:\Users\HP PRO\Downloads\block_detector_TRAINED.pt" "c:\Users\HP PRO\Desktop\sam_blocks_inventory\models\block_detector.pt"
   ```
3. **Deploy to Railway** (commit and push)
4. **Test on live site!**

## 📚 For Your Defense

**"How did you train the AI?"**
> "I used transfer learning with YOLOv8, starting from a model pretrained on 1.2M images. I fine-tuned it on my custom dataset of [X] labeled block images using Google Colab's free GPU. Training took 50 epochs with an 80/20 train-validation split. The model achieved [X]% mAP50 accuracy."

**"Why Google Colab?"**
> "Google Colab provides free GPU access (Tesla T4), which accelerates training 10-50x compared to CPU. It's industry-standard for ML training and eliminates the need for expensive hardware. Training that would take 4+ hours on CPU completed in 15-30 minutes on GPU."

**"What is mAP50?"**
> "Mean Average Precision at 50% IoU (Intersection over Union). It measures how accurately the model detects blocks. A score above 0.7 indicates the model correctly identifies blocks with good bounding box placement in most cases."

---

## 🛠️ Troubleshooting

**Out of memory error?**
- Reduce batch size: Change `batch=8` to `batch=4` in cell 8

**Low accuracy (< 0.5)?**
- Label 10-20 more images
- Train longer: Change `epochs=50` to `epochs=100`
- Use larger model: Change `yolov8n-seg.pt` to `yolov8s-seg.pt`

**Runtime disconnected?**
- All results are saved to Google Drive
- Rerun cells from the top
- Training will resume from last checkpoint

---

**🎉 Happy Training!**

---

## 🔍 TROUBLESHOOTING: 0 Blocks Detected

If your model detected 0 blocks, let's diagnose the problem!

In [ ]:
# Diagnostic 1: Check what's in your label files
print("🔍 DIAGNOSTIC 1: Inspecting Label Files")
print("=" * 60)

labels_path = Path('/content/block_dataset/labels')
label_files = list(labels_path.glob('*.txt'))

if label_files:
    # Check first 5 label files
    print(f"\n📄 Found {len(label_files)} label files")
    print("\nSample of first 3 labels:\n")
    
    for i, label_file in enumerate(label_files[:3]):
        print(f"File: {label_file.name}")
        with open(label_file, 'r') as f:
            content = f.read().strip()
            if content:
                lines = content.split('\n')
                print(f"  Lines: {len(lines)}")
                print(f"  First line: {lines[0]}")
                
                # Parse the line
                parts = lines[0].split()
                if len(parts) >= 5:
                    class_id = parts[0]
                    print(f"  Class ID: {class_id} (should be 0 for blocks)")
                    if class_id != '0':
                        print(f"  ⚠️  WARNING: Class ID is '{class_id}', not '0'!")
            else:
                print(f"  ⚠️  EMPTY FILE!")
        print()
    
    # Check if all labels use class 0
    all_classes = set()
    for label_file in label_files:
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    class_id = line.strip().split()[0]
                    all_classes.add(class_id)
    
    print(f"📊 All class IDs found in labels: {sorted(all_classes)}")
    if all_classes != {'0'}:
        print("⚠️  PROBLEM: Labels contain classes other than '0'!")
        print("   Your data.yaml says class 0 = block, but labels have different classes.")
else:
    print("❌ No label files found!")

print("\n" + "=" * 60)

In [ ]:
# Diagnostic 2: Visualize what the model learned
print("\n🔍 DIAGNOSTIC 2: Visualizing Training Data")
print("=" * 60)

from PIL import Image as PILImage
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Pick a training image with labels
train_images_path = Path('/content/block_dataset/train/images')
train_labels_path = Path('/content/block_dataset/train/labels')

train_imgs = list(train_images_path.glob('*'))
if train_imgs:
    sample_img = train_imgs[0]
    sample_label = train_labels_path / f"{sample_img.stem}.txt"
    
    print(f"📸 Sample training image: {sample_img.name}\n")
    
    # Load image
    img = PILImage.open(sample_img)
    img_width, img_height = img.size
    
    # Load labels
    if sample_label.exists():
        with open(sample_label, 'r') as f:
            labels = [line.strip().split() for line in f if line.strip()]
        
        print(f"📊 This image has {len(labels)} labeled object(s)")
        
        # Visualize
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        ax.imshow(img)
        
        # Draw bounding boxes from labels
        for i, label in enumerate(labels):
            class_id, x_center, y_center, width, height = map(float, label)
            
            # Convert normalized coordinates to pixels
            x_center_px = x_center * img_width
            y_center_px = y_center * img_height
            width_px = width * img_width
            height_px = height * img_height
            
            # Calculate top-left corner
            x1 = x_center_px - width_px / 2
            y1 = y_center_px - height_px / 2
            
            # Draw rectangle
            rect = patches.Rectangle(
                (x1, y1), width_px, height_px,
                linewidth=3, edgecolor='red', facecolor='none'
            )
            ax.add_patch(rect)
            
            # Add label
            ax.text(x1, y1-10, f'Class {int(class_id)}', 
                   color='red', fontsize=12, fontweight='bold',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax.set_title(f"Ground Truth Labels: {sample_img.name}", fontsize=14, fontweight='bold')
        ax.axis('off')
        plt.tight_layout()
        plt.show()
        
        print("\n💡 Questions to ask:")
        print("  1. Do the RED boxes match your hollow blocks?")
        print("  2. Or do they mark something else (bricks, walls, etc)?")
        print("  3. Are the boxes too small/large?")
        print("  4. Do they capture the full block or just part of it?")
    else:
        print(f"⚠️  No label file found for {sample_img.name}")
else:
    print("❌ No training images found!")

print("\n" + "=" * 60)

In [ ]:
# Diagnostic 3: Test on different confidence thresholds
print("\n🔍 DIAGNOSTIC 3: Testing Different Confidence Thresholds")
print("=" * 60)

# Test the same image with lower confidence threshold
test_image = list(val_images.glob('*'))[0]
print(f"Testing: {test_image.name}\n")

# Try different confidence thresholds
for conf_threshold in [0.001, 0.01, 0.05, 0.1, 0.25, 0.5]:
    results = trained_model(test_image, conf=conf_threshold, verbose=False)
    num_detections = len(results[0].boxes)
    print(f"Confidence {conf_threshold:.3f}: {num_detections} detection(s)")

print("\n💡 If you see 0 detections even at 0.001, the model didn't learn!")
print("=" * 60)

## 🔧 SOLUTIONS Based on Diagnosis

Run the diagnostic cells above, then choose your solution:

### Problem 1: Wrong Class IDs in Labels

**Symptom:** Diagnostic 1 shows class IDs other than '0'

**Cause:** Your labeling tool exported different class numbers

**Solution:** Fix all labels to use class 0

In [ ]:
# SOLUTION 1: Fix class IDs in all label files
# Run this ONLY if Diagnostic 1 showed wrong class IDs

import os

labels_dir = Path('/content/drive/MyDrive/block_dataset/labels')
fixed_count = 0

print("🔧 Fixing class IDs in label files...")
print("=" * 60)

for label_file in labels_dir.glob('*.txt'):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    new_lines = []
    modified = False
    
    for line in lines:
        if line.strip():
            parts = line.strip().split()
            if len(parts) >= 5:
                old_class = parts[0]
                if old_class != '0':
                    # Change class ID to 0
                    parts[0] = '0'
                    modified = True
                new_lines.append(' '.join(parts) + '\n')
            else:
                new_lines.append(line)
    
    if modified:
        with open(label_file, 'w') as f:
            f.writelines(new_lines)
        fixed_count += 1

print(f"✅ Fixed {fixed_count} label files")
print(f"📊 Total labels processed: {len(list(labels_dir.glob('*.txt')))}")
print("\n🔄 Now RE-RUN cells 5-8 to retrain with fixed labels!")
print("=" * 60)

### Problem 2: Labels Mark Wrong Objects

**Symptom:** Diagnostic 2 shows boxes around bricks/walls, not hollow blocks

**Cause:** Your labeling tool was trained on wrong images OR you labeled wrong objects

**Solution:** You need to re-label your images with the correct objects (hollow blocks only)

**How to fix:**

1. **Use LabelImg or Roboflow** to manually label hollow blocks:
   - Download LabelImg: https://github.com/HumanSignal/labelImg
   - Or use Roboflow: https://roboflow.com/ (easier, web-based)

2. **What to label:**
   - ✅ Hollow concrete blocks (interlocks)
   - ❌ NOT bricks, walls, or other objects

3. **Labeling tips:**
   - Draw tight boxes around each block
   - Include the full block (all edges)
   - Label ALL blocks in each image (don't skip any)
   - Class name: "block" or class ID: 0

4. **Export format:**
   - YOLO format (class x_center y_center width height)
   - All values normalized 0-1

5. **Upload corrected labels** back to Google Drive and re-train

### Problem 3: Not Enough Training Data

**Symptom:** Model trained but accuracy is very low (mAP50 < 0.3)

**Cause:** 39 images is borderline - needs more examples

**Solution:** Add more labeled images (aim for 50-100)

In [ ]:
# SOLUTION 3: Data augmentation (if you can't get more images)
# This artificially increases dataset by creating variations

from ultralytics import YOLO

print("🔧 Re-training with MORE augmentation...")
print("=" * 60)

# Retrain with aggressive augmentation
model_aug = YOLO('yolov8n-seg.pt')

results = model_aug.train(
    data=str(dataset_path / 'data.yaml'),
    epochs=100,             # More epochs!
    imgsz=640,
    batch=8,
    patience=20,            # More patience
    save=True,
    project=save_dir,
    name='sam_blocks_v2_augmented',
    exist_ok=True,
    
    # AGGRESSIVE AUGMENTATION
    hsv_h=0.05,            # Hue variation
    hsv_s=0.7,             # Saturation
    hsv_v=0.4,             # Brightness
    degrees=15,             # Rotation
    translate=0.2,          # Shifting
    scale=0.5,              # Zooming
    shear=10,               # Shearing
    perspective=0.001,      # Perspective transform
    flipud=0.5,             # Vertical flip
    fliplr=0.5,             # Horizontal flip
    mosaic=1.0,             # Mosaic augmentation
    mixup=0.1,              # Mixup augmentation
    
    verbose=True,
)

print("\n✅ Training complete with augmentation!")
print("🔄 Now run cells 9-11 again to test this new model")
print("=" * 60)